In [1]:
import numpy as np 
import pandas as pd
import pickle
from collections import defaultdict


In [3]:
data=pd.read_csv('AmazonDataset/1429_1.csv', low_memory=False)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

In [23]:
reviews=data[['asins','reviews.date','reviews.text']].rename(columns= {'asins':'product','reviews.date':'year','reviews.text':'review'})
reviews

,product,year,review
0,B01AHB9CN2,2017-01-13T00:00:00.000Z,This product so far has not disappointed. My c...
1,B01AHB9CN2,2017-01-13T00:00:00.000Z,great for beginner or experienced person. Boug...
2,B01AHB9CN2,2017-01-13T00:00:00.000Z,Inexpensive tablet for him to use and learn on...
3,B01AHB9CN2,2017-01-13T00:00:00.000Z,I've had my Fire HD 8 two weeks now and I love...
4,B01AHB9CN2,2017-01-12T00:00:00.000Z,I bought this for my grand daughter when she c...
...,...,...,...
34655,B006GWO5WK,2012-09-18T00:00:00Z,This is not appreciably faster than any other ...
34656,B006GWO5WK,2012-11-21T00:00:00Z,Amazon should include this charger with the Ki...
34657,B006GWO5WK,2012-10-19T00:00:00Z,Love my Kindle Fire but I am really disappoint...
34658,B006GWO5WK,2012-10-31T00:00:00Z,I was surprised to find it did not come with a...


In [24]:
reviews =reviews.dropna()
reviews

,product,year,review
0,B01AHB9CN2,2017-01-13T00:00:00.000Z,This product so far has not disappointed. My c...
1,B01AHB9CN2,2017-01-13T00:00:00.000Z,great for beginner or experienced person. Boug...
2,B01AHB9CN2,2017-01-13T00:00:00.000Z,Inexpensive tablet for him to use and learn on...
3,B01AHB9CN2,2017-01-13T00:00:00.000Z,I've had my Fire HD 8 two weeks now and I love...
4,B01AHB9CN2,2017-01-12T00:00:00.000Z,I bought this for my grand daughter when she c...
...,...,...,...
34655,B006GWO5WK,2012-09-18T00:00:00Z,This is not appreciably faster than any other ...
34656,B006GWO5WK,2012-11-21T00:00:00Z,Amazon should include this charger with the Ki...
34657,B006GWO5WK,2012-10-19T00:00:00Z,Love my Kindle Fire but I am really disappoint...
34658,B006GWO5WK,2012-10-31T00:00:00Z,I was surprised to find it did not come with a...


In [25]:
reviews['year']=reviews['year'].str[:4].astype(int)
reviews

/tmp/ipykernel_1231329/313924133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['year']=reviews['year'].str[:4].astype(int)


,product,year,review
0,B01AHB9CN2,2017,This product so far has not disappointed. My c...
1,B01AHB9CN2,2017,great for beginner or experienced person. Boug...
2,B01AHB9CN2,2017,Inexpensive tablet for him to use and learn on...
3,B01AHB9CN2,2017,I've had my Fire HD 8 two weeks now and I love...
4,B01AHB9CN2,2017,I bought this for my grand daughter when she c...
...,...,...,...
34655,B006GWO5WK,2012,This is not appreciably faster than any other ...
34656,B006GWO5WK,2012,Amazon should include this charger with the Ki...
34657,B006GWO5WK,2012,Love my Kindle Fire but I am really disappoint...
34658,B006GWO5WK,2012,I was surprised to find it did not come with a...


In [34]:
reviews.sort_values(by='product').sort_values(by='year')

,product,year,review
2898,B002Y27P3M,2010,I researched the purchase of a Kindle for a lo...
34636,B002Y27P6Y,2010,I found this cover to be quite handsome in app...
34638,B002Y27P6Y,2010,This cover is a replacement for the initial co...
34635,B002Y27P6Y,2010,I have this cover in black and my cover does i...
34637,B002Y27P6Y,2010,"I, as well as other Kindle lovers, have notice..."
...,...,...,...
29535,B01BH83OOM,2018,Loud enough for small gatherings & very respon...
29331,B01BH83OOM,2018,Great speaker for around the house listening t...
29469,B01BH83OOM,2018,I spent more time trying get it functioning th...
29008,B01BH83OOM,2018,Took a bit of time but I figured how to get th...


In [35]:
with open('Reviews_DataFrame.pkl', 'wb') as f:
    pickle.dump(reviews, f)

reviews

,product,year,review
0,B01AHB9CN2,2017,This product so far has not disappointed. My c...
1,B01AHB9CN2,2017,great for beginner or experienced person. Boug...
2,B01AHB9CN2,2017,Inexpensive tablet for him to use and learn on...
3,B01AHB9CN2,2017,I've had my Fire HD 8 two weeks now and I love...
4,B01AHB9CN2,2017,I bought this for my grand daughter when she c...
...,...,...,...
34655,B006GWO5WK,2012,This is not appreciably faster than any other ...
34656,B006GWO5WK,2012,Amazon should include this charger with the Ki...
34657,B006GWO5WK,2012,Love my Kindle Fire but I am really disappoint...
34658,B006GWO5WK,2012,I was surprised to find it did not come with a...
